<a href="https://colab.research.google.com/github/okita-shota/projectA_G/blob/main/YOLOv8_HumanPoseEstimation_%E3%81%AE10_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOで人体姿勢推定



In [ ]:
#Google Colab ngrok
!pip install flask pyngrok

In [18]:
#ngrok
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'



In [19]:
#flask

from pyngrok import ngrok

public_url = ngrok.connect(8080)
print('WebアプリのURL:', public_url)

ERROR:pyngrok.process.ngrok:t=2024-11-04T16:18:42+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-04T16:18:42+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-04T16:18:42+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

## Y00. セットアップ

In [ ]:
!nvidia-smi

Sat Nov  2 15:45:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Y01. YOLOv8をインストール




In [ ]:
# 1 YOLOv8をダウンロード
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.1/112.6 GB disk)


In [ ]:
# 2 必要なライブラリのインストール
import csv
import cv2
from ultralytics import YOLO

In [ ]:
# 3 学習モデルの読み込み。姿勢推論用のモデルデータを読み込む
model = YOLO("yolov8n-pose.pt")

100%|██████████| 6.52M/6.52M [00:00<00:00, 101MB/s]


In [ ]:
# 4 keypointの位置毎の名称定義
KEYPOINTS_NAMES = [
    "nose", "eye(L)", "eye(R)", "ear(L)", "ear(R)",
    "shoulder(L)", "shoulder(R)", "elbow(L)", "elbow(R)",
    "wrist(L)", "wrist(R)", "hip(L)", "hip(R)",
    "knee(L)", "knee(R)", "ankle(L)", "ankle(R)"
]

In [ ]:
# 5 ビデオライターの設定
def setup_video_writer(capture, output_path):
    """ビデオライターの設定"""
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = capture.get(cv2.CAP_PROP_FPS)
    width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
# 6 姿勢情報をCSVファイルに書き出す
def write_pose_to_csv(csv_path, frame_count, keypoints, confs):
    """姿勢情報をCSVファイルに書き出す"""
    row = [frame_count]
    for index, keypoint in enumerate(zip(keypoints, confs)):
        x, y = int(keypoint[0][0]), int(keypoint[0][1])
        score = keypoint[1]
        row.extend([x, y, score])
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

In [ ]:
# 7 フレームにキーポイントと骨格を描画する
def draw_keypoints(frame, keypoints, confs):
    """フレームにキーポイントと骨格を描画する"""
    for index, keypoint in enumerate(zip(keypoints, confs)):
        x, y = int(keypoint[0][0]), int(keypoint[0][1])
        score = keypoint[1]
        if score >= 0.5:
            cv2.circle(frame, (x, y), 5, (255, 0, 255), -1)
            cv2.putText(frame, KEYPOINTS_NAMES[index], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1, cv2.LINE_AA)
    return frame


In [ ]:
# 8 動画をフレームごとに処理し、姿勢情報を取得してCSVと動画に保存する
def process_video(input_video_path, output_video_path, csv_path):
    """動画をフレームごとに処理し、姿勢情報を取得してCSVと動画に保存する"""
    capture = cv2.VideoCapture(input_video_path)
    video_writer = setup_video_writer(capture, output_video_path)

    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # ヘッダー行を書き込む
        header = ["frame"]
        for name in KEYPOINTS_NAMES:
            header.extend([f"{name}_x", f"{name}_y", f"{name}_score"])
        writer.writerow(header)

    frame_count = 0

    while capture.isOpened():
        success, frame = capture.read()
        if not success:
            break

        # 推論を実行
        results = model(frame)

        if len(results[0].keypoints) > 0:
            keypoints = results[0].keypoints
            confs = keypoints.conf[0].tolist()  # 推論結果:1に近いほど信頼度が高い
            xys = keypoints.xy[0].tolist()  # 座標

            # 姿勢情報をCSVファイルに書き出す
            write_pose_to_csv(csv_path, frame_count, xys, confs)
            # キーポイントと骨格をフレームに描画する
            frame = draw_keypoints(frame, xys, confs)

        # フレームに骨格情報を描画したものを動画に書き出す
        video_writer.write(frame)
        frame_count += 1

    capture.release()
    video_writer.release()
    cv2.destroyAllWindows()

## Y02. サンプル動画（sample.mp4）の読み込み

In [ ]:
# Y02.1　（一番簡単）
# ファイルをドラッグ＆ドロップ '/content/sample.mp4'

# Y02-.2
# 解析用動画のアップロード（無料アカウントの場合はエラー）
# from google.colab import files
# uploaded = files.upload()

## Y03. サンプル動画（sample.mp4）の骨格情報の推定

### Y03.1 （今回は）fpsを30で統一する

In [ ]:
### Y03.1 今回はfpsを30で統一する

# ドラッグ&ドロップの場合
#!ffmpeg -y -i "sample.mp4" -vf "fps=30" "sample_30fps.mp4"

# # Google Driveの場合
!ffmpeg -y -i "/content/drive/MyDrive//content//content/46894_640x360ストレッチ高齢者2.mp4/sample.mp4" -vf "fps=30" "sample_30fps.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Y03.2 動画のwidth/height/fpsを取得

In [ ]:
### Y03.2 動画のwidth/height/fpsを取得
import cv2

# ドラッグ&ドロップの場合
# video_path = "sample.mp4"

# # Google Driveの場合
video_path = "/content/drive/MyDrive/20240903_video/sample.mp4"

cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("sample.mp4")
print("width:{}, height:{}, fps:{}".format(width,height,fps))

video_path = "sample_30fps.mp4"
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("")
print("sample_30fps.mp4")
print("width:{}, height:{}, fps:{}".format(width,height,fps))

sample.mp4
width:0.0, height:0.0, fps:0.0

sample_30fps.mp4
width:0.0, height:0.0, fps:0.0


### Y03.3 入力動画のパスと出力ファイルのパスを指定

In [ ]:
### Y03.3 入力動画のパスと出力ファイルのパスを指定
input_video_path = '/content/sample_30fps.mp4'
output_video_path = 'sample_30fps_with_pose.mp4'
csv_path = 'sample_30fps_pose_keypoints.csv'

### Y03.4 動画を処理し、姿勢情報を取得してCSVと動画に保存

In [ ]:
### Y03.4 動画を処理し、姿勢情報を取得してCSVと動画に保存
process_video(input_video_path, output_video_path, csv_path)

### Y03.5 結果確認用動画の作成

In [ ]:
### Y03.5 結果確認用動画の作成
!ffmpeg -y -i {output_video_path} -vf scale=600:-2 {"sample_30fps_with_pose.mov"}

### Y03.6 動画の再生

In [ ]:
### Y03.6 動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open( "sample_30fps_with_pose.mov", 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")


## Y04. 比較用動画（f0.mp4）の読み込み

In [ ]:
# Y04.1
# ドラッグ＆ドロップ '/content/f0.mp4'

# Y04.2
# 解析したい動画のアップロード （無料アカウントの場合はエラー）
# from google.colab import files
# uploaded = files.upload()

## Y05. 比較用動画（f0.mp4）の骨格情報の推定

### Y05.1 （今回は）fpsを30で統一する

In [ ]:
### Y05.1 今回はfpsを30で統一する
# sample.mp4を変換した時と比べて、どこが変更されているか要確認。 答え：ファイル名だけです。
# ドラッグ&ドロップの場合
# !ffmpeg -y -i "f0.mp4" -vf "fps=30" "f0_30fps.mp4"

# # Google Driveの場合
!ffmpeg -y -i "/content/drive/MyDrive/20240903_video/f0.mp4" -vf "fps=30" "f0_30fps.mp4"

### Y05.2 動画のwidth/height/fpsを取得

In [ ]:
### Y05.2 動画のwidth/height/fpsを取得
import cv2

# # ドラッグ&ドロップの場合
# video_path = "f0.mp4"
# # Google Driveの場合
video_path = "/content/drive/MyDrive/20240903_video/f0.mp4"

cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("f0.mp4")
print("width:{}, height:{}, fps:{}".format(width,height,fps))

video_path = "f0_30fps.mp4"
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("")
print("f0_30fps.mp4")
print("width:{}, height:{}, fps:{}".format(width,height,fps))

### Y05.3 入力動画のパスと出力ファイルのパスを指定

In [ ]:
### Y05.3 入力動画のパスと出力ファイルのパスを指定
input_video_path = '/content/f0_30fps.mp4'
output_video_path = 'f0_30fps_with_pose.mp4'
csv_path = 'f0_30fps_pose_keypoints.csv'

### Y05.4 動画を処理し、姿勢情報を取得してCSVと動画に保存

In [ ]:
### Y05.4 動画を処理し、姿勢情報を取得してCSVと動画に保存
process_video(input_video_path, output_video_path, csv_path)

### Y05.5 結果確認用動画の作成

In [ ]:
### Y05.5 結果確認用動画の作成
!ffmpeg -y -i {output_video_path} -vf scale=600:-2 {"f0_30fps_with_pose.mov"}

### Y05.6 動画の再生

In [ ]:
### Y05.6 動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open( "f0_30fps_with_pose.mov", 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")


### Y05.7 他の動画（f1.mp4）でも適用してみよう

In [ ]:
# Y05.7.1
# f1.mp4をアップロードしよう

In [ ]:
# Y05.7.2
# 簡単にするためにファイル名と拡張子を最初に定義
file_name = "f1"
extension = ".mp4"

### Y05.1 今回はfpsを30で統一する
# # ドラッグ&ドロップの場合
# !ffmpeg -y -i {file_name}{extension} -vf "fps=30" {file_name}"_30fps.mp4"
# # Google Driveの場合
!ffmpeg -y -i "/content/drive/MyDrive/20240903_video/"{file_name}{extension} -vf "fps=30" {file_name}"_30fps.mp4"

### Y05.2 動画のwidth/height/fpsを取得
import cv2

video_path = file_name+extension
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

print(file_name+extension)
print("width:{}, height:{}, fps:{}".format(width,height,fps))

video_path = file_name+"_30fps.mp4"
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("")
print(file_name+"_30fps.mp4")
print("width:{}, height:{}, fps:{}".format(width,height,fps))

### Y05.3 入力動画のパスと出力ファイルのパスを指定
input_video_path = "/content/"+file_name+"_30fps.mp4"
output_video_path = file_name+"_30fps_with_pose.mp4"
csv_path = file_name+"_30fps_pose_keypoints.csv"

### Y05.4 動画を処理し、姿勢情報を取得してCSVと動画に保存
process_video(input_video_path, output_video_path, csv_path)

### Y05.5 結果確認用動画の作成
!ffmpeg -y -i {output_video_path} -vf scale=600:-2 {file_name}_30fps_with_pose.mov

### Y05.6 動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open( file_name+"_30fps_with_pose.mov", 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")



In [ ]:
# Y05.7.3
# 関数として定義しておく

def get_pose(file_name:str, extension:str):
  ### Y05.1 今回はfpsを30で統一する
  # # ドラッグ&ドロップの場合
  # !ffmpeg -y -i {file_name}{extension} -vf "fps=30" {file_name}"_30fps.mp4"
  # # Google Driveの場合
  !ffmpeg -y -i "/content/drive/MyDrive/20240903_video/"{file_name}{extension} -vf "fps=30" {file_name}"_30fps.mp4"

  ### Y05.2 動画のwidth/height/fpsを取得
  import cv2

  video_path = file_name+extension
  cap = cv2.VideoCapture(video_path)
  width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fps = cap.get(cv2.CAP_PROP_FPS)

  print(file_name+extension)
  print("width:{}, height:{}, fps:{}".format(width,height,fps))

  video_path = file_name+"_30fps.mp4"
  cap = cv2.VideoCapture(video_path)
  width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fps = cap.get(cv2.CAP_PROP_FPS)

  print("")
  print(file_name+"_30fps.mp4")
  print("width:{}, height:{}, fps:{}".format(width,height,fps))

  ### Y05.3 入力動画のパスと出力ファイルのパスを指定
  input_video_path = "/content/"+file_name+"_30fps.mp4"
  output_video_path = file_name+"_30fps_with_pose.mp4"
  csv_path = file_name+"_30fps_pose_keypoints.csv"

  ### Y05.4 動画を処理し、姿勢情報を取得してCSVと動画に保存
  process_video(input_video_path, output_video_path, csv_path)

  ### Y05.5 結果確認用動画の作成
  !ffmpeg -y -i {output_video_path} -vf scale=600:-2 {file_name}_30fps_with_pose.mov

  ### Y05.6 動画の再生
  from IPython.display import HTML
  from base64 import b64encode

  mp4 = open( file_name+"_30fps_with_pose.mov", 'rb').read()
  data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
  HTML(f"""
  <video width="50%" height="50%" controls>
        <source src="{data_url}" type="video/mp4">
  </video>""")

In [ ]:
# Y05.7.4
# f2.mp4をアップロードしてみよう

In [ ]:
# Y05.7.5
# これ以降はこの関数を呼び出すだけで、骨格推定が実行できる
get_pose("f2", ".mp4")

## Y06. 単位ベクトルに変換

### Y06.1 骨格情報が入ったcsvファイルの中身を確認

In [ ]:
### Y06.1 骨格情報が入ったcsvファイルの中身を確認
import pandas as pd

pd.read_csv("sample_30fps_pose_keypoints.csv").head()

### Y06.2 0フレーム目を可視化


In [ ]:
### Y06.2 0フレーム目を可視化
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def visualize_zero_frame(file_path:str):
  data = pd.read_csv(file_path)

  # 0フレームのデータをフィルタリングする
  frame_0_data = data[data['frame'] == 0]

  # キーポイントの名前のリスト（列名からスコアを除いたもの）
  keypoint_names = [col.split('_')[0] for col in frame_0_data.columns[1::3]]

  # 0フレームのすべてのキーポイントのx座標とy座標を抽出する
  x_coords = frame_0_data.iloc[0, 1::3]
  y_coords = frame_0_data.iloc[0, 2::3]

  # キーポイントの接続を定義（CSVのラベルに基づく）
  keypoints = {
      'nose': 0, 'eye(L)': 1, 'eye(R)': 2, 'ear(L)': 3, 'ear(R)': 4,
      'shoulder(L)': 5, 'shoulder(R)': 6, 'elbow(L)': 7, 'elbow(R)': 8,
      'wrist(L)': 9, 'wrist(R)': 10, 'hip(L)': 11, 'hip(R)': 12,
      'knee(L)': 13, 'knee(R)': 14, 'ankle(L)': 15, 'ankle(R)': 16
  }

  connections = [
      ('nose', 'eye(L)'), ('nose', 'eye(R)'), ('eye(L)', 'ear(L)'), ('eye(R)', 'ear(R)'),
      ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
      ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
      ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
      ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
      ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
  ]

  # y軸を反転させた状態で0フレームの散布図をプロットする
  plt.figure(figsize=(10, 8))

  # キーポイントをプロット
  plt.scatter(x_coords, y_coords, c='blue')

  # 各キーポイントの横に名前をプロットする
  for i, name in enumerate(keypoint_names):
      plt.text(x_coords[i], y_coords[i], name, fontsize=9, ha='right')

  # キーポイントを矢印で接続
  for connection in connections:
      x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
      x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
      dx, dy = x2 - x1, y2 - y1
      plt.arrow(x1, y1, dx, dy, head_width=15, head_length=25, fc='gray', ec='gray', length_includes_head=True)

  plt.xlabel('X Coordinates')
  plt.ylabel('Y Coordinates')
  plt.title('Scatter Plot of Keypoints for Frame 0')
  plt.grid(True)
  plt.gca().invert_yaxis()  # 原点を左上にするためにy軸を反転する
  plt.axis('equal')  # 縦軸と横軸のスケールを同じにする
  plt.show()


In [ ]:
visualize_zero_frame('sample_30fps_pose_keypoints.csv')

### Y06.3 0フレーム目を単位ベクトルに変換（可視化確認用）

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def visualize_zero_frame_norm(file_path:str):
  data = pd.read_csv(file_path)

  # 0フレームのデータをフィルタリングする
  frame_0_data = data[data['frame'] == 0]

  # キーポイントの名前のリスト（列名からスコアを除いたもの）
  keypoint_names = [col.split('_')[0] for col in frame_0_data.columns[1::3]]

  # 0フレームのすべてのキーポイントのx座標とy座標を抽出する
  x_coords = frame_0_data.iloc[0, 1::3]
  y_coords = frame_0_data.iloc[0, 2::3]

  # キーポイントの接続を定義（CSVのラベルに基づく）
  keypoints = {
      'nose': 0, 'eye(L)': 1, 'eye(R)': 2, 'ear(L)': 3, 'ear(R)': 4,
      'shoulder(L)': 5, 'shoulder(R)': 6, 'elbow(L)': 7, 'elbow(R)': 8,
      'wrist(L)': 9, 'wrist(R)': 10, 'hip(L)': 11, 'hip(R)': 12,
      'knee(L)': 13, 'knee(R)': 14, 'ankle(L)': 15, 'ankle(R)': 16
  }

  connections = [
      # ('nose', 'eye(L)'), ('nose', 'eye(R)'), ('eye(L)', 'ear(L)'), ('eye(R)', 'ear(R)'),
      ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
      ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
      ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
      ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
      ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
  ]

  # y軸を反転させた状態で0フレームの散布図をプロットする
  plt.figure(figsize=(10, 8))

  # キーポイントをプロット
  plt.scatter(x_coords, y_coords, c='blue')

  # 各キーポイントの横に名前をプロットする
  for i, name in enumerate(keypoint_names):
      plt.text(x_coords[i], y_coords[i], name, fontsize=9, ha='right')

  # キーポイントを矢印で接続
  for connection in connections:
      x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
      x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
      dx, dy = x2 - x1, y2 - y1
      plt.arrow(x1, y1, dx, dy, head_width=15, head_length=25, fc='gray', ec='gray', length_includes_head=True)

  # 単位ベクトルを計算する
  unit_vectors = []
  for connection in connections:
      x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
      x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
      dx, dy = x2 - x1, y2 - y1
      norm = np.sqrt(dx**2 + dy**2)
      if norm != 0:
          unit_vectors.append((dx / norm, dy / norm, x1, y1))

  # 単位ベクトルをプロットする
  for uv in unit_vectors:
      plt.arrow(uv[2], uv[3], uv[0] * 100, uv[1] * 100, head_width=5, head_length=10, fc='red', ec='red', length_includes_head=True)

  plt.xlabel('X Coordinates')
  plt.ylabel('Y Coordinates')
  plt.title('Visualization of Unit Vectors for Frame 0')
  plt.grid(True)
  plt.gca().invert_yaxis()  # 原点を左上にするためにy軸を反転する
  plt.axis('equal')  # 縦軸と横軸のスケールを同じにする
  plt.show()


In [ ]:
visualize_zero_frame_norm( 'sample_30fps_pose_keypoints.csv')

### Y06.4 全てのフレームを単位ベクトルに変換してcsvファイルに格納

In [ ]:
### Y06.4 全てのフレームを単位ベクトルに変換してcsvファイルに格納

import pandas as pd
import numpy as np

# 単位ベクトルを計算し、結果を連結する関数を定義
def calc_and_concat_norms(cols, data):
    unit_vectors_list = []
    for col1, col2 in cols:
        # 各点のconfidenceを取得
        confidence_col1 = data[f'{col1}_score'].values
        confidence_col2 = data[f'{col2}_score'].values

        # ベクトルの差分を計算し、confidenceが0.1以下の場合は差分を0に設定
        vector_diff = np.where((confidence_col1 > 0.1)[:, None] & (confidence_col2 > 0.1)[:, None],
                               data[[f'{col2}_x', f'{col2}_y']].values - data[[f'{col1}_x', f'{col1}_y']].values,
                               np.array([0, 0]))

        # 単位ベクトルを計算
        norm_diff = np.linalg.norm(vector_diff, axis=1, keepdims=True)
        unit_vector_diff = np.where(norm_diff != 0, vector_diff / norm_diff, np.array([0, 0]))

        # 結果をデータフレームに変換
        unit_vectors_df = pd.DataFrame({
            f'unit_x_{col1}_{col2}': unit_vector_diff[:, 0],
            f'unit_y_{col1}_{col2}': unit_vector_diff[:, 1]
        })

        # 各ペアの結果をリストに追加
        unit_vectors_list.append(unit_vectors_df)

    # 全てのデータフレームを列方向に連結
    final_df = pd.concat(unit_vectors_list, axis=1)
    return final_df

# 指定されたディレクトリ内のファイルを処理する関数を定義
def transform_to_norm(filename: str):
    # ファイルパスを設定
    file_path = f"{filename}_30fps_pose_keypoints.csv"
    output_file_path = f"{filename}_30fps_pose_keypoints_norm.csv"

    # CSVファイルを読み込む
    data = pd.read_csv(file_path).fillna(0)

    # 対象の点のペアを定義（0フレーム目の可視化に基づく）
    points_pairs = [
        # ('nose', 'eye(L)'), ('nose', 'eye(R)'), ('eye(L)', 'ear(L)'), ('eye(R)', 'ear(R)'),
        ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
        ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
        ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
        ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
        ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
    ]

    # 単位ベクトルを計算し連結
    result_df = calc_and_concat_norms(points_pairs, data)

    # 結果をCSVファイルに保存
    result_df.to_csv(output_file_path, index=False)



In [ ]:
# 指定されたディレクトリで関数を実行
transform_to_norm("sample")
transform_to_norm("f0")
transform_to_norm("f1")
transform_to_norm("f2")

### Y06.5 変換結果のcsvファイルの中身を確認

In [ ]:
### Y06.5 変換結果のcsvファイルの中身を確認
import pandas as pd

pd.read_csv("sample_30fps_pose_keypoints_norm.csv").head()

### Y06.5 変換結果の可視化

In [ ]:
### Y06.5 変換結果の可視化

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

def visualize_norms(file_name: str):
    # CSVファイルを読み込む
    file_path = file_name + '_30fps_pose_keypoints.csv'
    data = pd.read_csv(file_path)

    # 単位ベクトルが計算された結果のCSVファイルを読み込む
    norm_file_path = file_name + '_30fps_pose_keypoints_norm.csv'
    norm_data = pd.read_csv(norm_file_path)

    # フレーム数を取得
    num_frames = norm_data.shape[0]

    # カラーマップを設定（半透明にする）
    colors = cm.viridis(np.linspace(0, 1, num_frames))
    colors[:, 3] = 0.2  # アルファ値を設定（0.5にする）

    # 初期フレームのデータをフィルタリングする
    frame_0_data = data[data['frame'] == 0]

    # キーポイントの名前のリスト（列名からスコアを除いたもの）
    keypoint_names = [col.split('_')[0] for col in frame_0_data.columns[1::3]]

    # 初期フレームのすべてのキーポイントのx座標とy座標を抽出する
    x_coords = frame_0_data.iloc[0, 1::3]
    y_coords = frame_0_data.iloc[0, 2::3]

    # キーポイントの接続を定義（CSVのラベルに基づく）
    keypoints = {
        'nose': 0, 'eye(L)': 1, 'eye(R)': 2, 'ear(L)': 3, 'ear(R)': 4,
        'shoulder(L)': 5, 'shoulder(R)': 6, 'elbow(L)': 7, 'elbow(R)': 8,
        'wrist(L)': 9, 'wrist(R)': 10, 'hip(L)': 11, 'hip(R)': 12,
        'knee(L)': 13, 'knee(R)': 14, 'ankle(L)': 15, 'ankle(R)': 16
    }

    connections = [
        ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
        ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
        ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
        ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
        ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
    ]

    # y軸を反転させた状態で初期フレームの散布図をプロットする
    plt.figure(figsize=(10, 8))

    # 初期フレームのキーポイントをプロット
    plt.scatter(x_coords, y_coords, c='blue')

    # 各キーポイントの横に名前をプロットする
    for i, name in enumerate(keypoint_names):
        plt.text(x_coords[i], y_coords[i], name, fontsize=9, ha='right')

    # 初期フレームのキーポイントを矢印で接続
    for connection in connections:
        x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
        x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
        dx, dy = x2 - x1, y2 - y1
        plt.arrow(x1, y1, dx, dy, head_width=15, head_length=25, fc='gray', ec='gray', length_includes_head=True)

    # 単位ベクトルをプロットする
    for connection in connections:
        unit_x_col = f'unit_x_{connection[0]}_{connection[1]}'
        unit_y_col = f'unit_y_{connection[0]}_{connection[1]}'
        if unit_x_col in norm_data.columns and unit_y_col in norm_data.columns:
            for i in range(num_frames):
                x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
                unit_x, unit_y = norm_data.iloc[i][unit_x_col], norm_data.iloc[i][unit_y_col]
                plt.arrow(x1, y1, unit_x * 100, unit_y * 100, head_width=5, head_length=10, fc=colors[i], ec=colors[i], length_includes_head=True)

    plt.xlabel('X Coordinates')
    plt.ylabel('Y Coordinates')
    plt.title('Visualization of Unit Vectors Over Time')
    plt.grid(True)
    plt.gca().invert_yaxis()  # 原点を左上にするためにy軸を反転する
    plt.axis('equal')  # 縦軸と横軸のスケールを同じにする
    plt.show()


In [ ]:
visualize_norms("sample")

# Y07. 動的時間伸縮法 / DTW (Dynamic Time Warping) を計算

<img src="https://i.gyazo.com/7cd1e5f76b8dd0f8db21792cfd09f517.gif" alt="GIF Image">


In [ ]:
### Y07.1 ライブラリをインストール
!pip install fastdtw

In [ ]:
### Y07.2 DTWの計算
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def calc_dtw(a, b):
  x = np.array(pd.read_csv(a).fillna(0))
  y = np.array(pd.read_csv(b).fillna(0))
  distance, path = fastdtw(x, y, dist=euclidean)
  print(a, b, distance)
 # return distance
calc_dtw("sample_30fps_pose_keypoints_norm.csv","f0_30fps_pose_keypoints_norm.csv")
calc_dtw("sample_30fps_pose_keypoints_norm.csv","f1_30fps_pose_keypoints_norm.csv")
calc_dtw("sample_30fps_pose_keypoints_norm.csv","f2_30fps_pose_keypoints_norm.csv")

DTWは時間軸方向を圧縮しているため、リズムに合わせるなどは無視してしまっている点に注意！

# Y08. 二乗平均平方根誤差 / RMSE(Root Mean Squared Error)を計算する

<img src="https://tsuchidalab.jp/wp1/wp-content/uploads/2024/05/1000.png" alt="Image">


In [ ]:
### Y08.2 Root Mean Squared Error(RMSE)の計算
# from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def calc_rmse(a, b):
  x = pd.read_csv(a).fillna(0)
  y = pd.read_csv(b).fillna(0)[:len(x)] #元動画の長さに合わせる
  rmse = np.sqrt(mean_squared_error(x, y))
  print(a, b,'{:.3f}'.format(rmse))

calc_rmse("sample_30fps_pose_keypoints_norm.csv","f0_30fps_pose_keypoints_norm.csv")
calc_rmse("sample_30fps_pose_keypoints_norm.csv","f1_30fps_pose_keypoints_norm.csv")
calc_rmse("sample_30fps_pose_keypoints_norm.csv","f2_30fps_pose_keypoints_norm.csv")

厳密にチェックしているけど、ちょっとでもリズムを取るタイミングがズレると、動きが似ていても大きな値を取ってしまう

# Y09 オリジナル動画を読み込んでみよう

## オリジナル一つ目の動画をアップロード

In [ ]:
# 動画の長さは以下のコマンドを使って適宜編集しよう
# スマホ上で編集するのが一番楽だとは思います（注：研究評価など厳密なスコアを算出する際には楽曲ベースで正確に切り出すこと）
# 動画の縦横は気にしなくて良いです

uploadfile = "test.mov"
# 開始10秒後から30秒間の動画を切り出したい場合、次のようにコマンドを入力
# !ffmpeg -ss 10 -i {uploadfile} -t 30 -c copy test.mp4

In [ ]:
## 動画は"/content/drive/MyDrive/20240903_video/" にアップロードすること。じゃないと動きません（パスの話）

### 骨格推定の適用
fname = "test" # ここの名前をアップロードした名前で。楽なのはこれに合わせてtestという名前の動画にしておく。

# ファイル名は適宜アップロードした内容と合わせること
get_pose(fname, ".mp4") # ここの拡張子もアップロードした名前で。iPhoneならMOVになると思います。

### 推定結果の確認
import pandas as pd

pd.read_csv(fname + "_30fps_pose_keypoints.csv").head()

# ゼロフレーム目の可視化
visualize_zero_frame(fname + '_30fps_pose_keypoints.csv')

# ゼロフレーム目の単位ベクトルの可視化
visualize_zero_frame_norm(fname + '_30fps_pose_keypoints.csv')

# 単位ベクトル作成
transform_to_norm(fname)

# 単位ベクトルの可視化
visualize_norms(fname)

## 課題動画

In [ ]:
### 骨格推定の適用
fname = "mihon"

# ファイル名は適宜アップロードした内容と合わせること
get_pose(fname, ".mp4")

### 推定結果の確認
import pandas as pd

pd.read_csv(fname + "_30fps_pose_keypoints.csv").head()

# ゼロフレーム目の可視化
visualize_zero_frame(fname + '_30fps_pose_keypoints.csv')

# ゼロフレーム目の単位ベクトルの可視化
visualize_zero_frame_norm(fname + '_30fps_pose_keypoints.csv')

# 単位ベクトル作成
transform_to_norm(fname)

# 単位ベクトルの可視化
visualize_norms(fname)

# DTWコスト計算

In [ ]:
### DTWコスト計算

calc_dtw("test_30fps_pose_keypoints_norm.csv","mihon_30fps_pose_keypoints_norm.csv")

# 以下はdef関連のエラーが出た人用。以下のコードを実行後に上の課題用コードに戻ってください。

In [ ]:
# 1 YOLOv8をダウンロード
%pip install ultralytics
import ultralytics
ultralytics.checks()

# 2 必要なライブラリのインストール
import csv
import cv2
from ultralytics import YOLO

# 3 学習モデルの読み込み。姿勢推論用のモデルデータを読み込む
model = YOLO("yolov8n-pose.pt")

# 4 keypointの位置毎の名称定義
KEYPOINTS_NAMES = [
    "nose", "eye(L)", "eye(R)", "ear(L)", "ear(R)",
    "shoulder(L)", "shoulder(R)", "elbow(L)", "elbow(R)",
    "wrist(L)", "wrist(R)", "hip(L)", "hip(R)",
    "knee(L)", "knee(R)", "ankle(L)", "ankle(R)"
]

# 5 ビデオライターの設定
def setup_video_writer(capture, output_path):
    """ビデオライターの設定"""
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = capture.get(cv2.CAP_PROP_FPS)
    width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 6 姿勢情報をCSVファイルに書き出す
def write_pose_to_csv(csv_path, frame_count, keypoints, confs):
    """姿勢情報をCSVファイルに書き出す"""
    row = [frame_count]
    for index, keypoint in enumerate(zip(keypoints, confs)):
        x, y = int(keypoint[0][0]), int(keypoint[0][1])
        score = keypoint[1]
        row.extend([x, y, score])
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

# 7 フレームにキーポイントと骨格を描画する
def draw_keypoints(frame, keypoints, confs):
    """フレームにキーポイントと骨格を描画する"""
    for index, keypoint in enumerate(zip(keypoints, confs)):
        x, y = int(keypoint[0][0]), int(keypoint[0][1])
        score = keypoint[1]
        if score >= 0.5:
            cv2.circle(frame, (x, y), 5, (255, 0, 255), -1)
            cv2.putText(frame, KEYPOINTS_NAMES[index], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1, cv2.LINE_AA)
    return frame

# 8 動画をフレームごとに処理し、姿勢情報を取得してCSVと動画に保存する
def process_video(input_video_path, output_video_path, csv_path):
    """動画をフレームごとに処理し、姿勢情報を取得してCSVと動画に保存する"""
    capture = cv2.VideoCapture(input_video_path)
    video_writer = setup_video_writer(capture, output_video_path)

    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # ヘッダー行を書き込む
        header = ["frame"]
        for name in KEYPOINTS_NAMES:
            header.extend([f"{name}_x", f"{name}_y", f"{name}_score"])
        writer.writerow(header)

    frame_count = 0

    while capture.isOpened():
        success, frame = capture.read()
        if not success:
            break

        # 推論を実行
        results = model(frame)

        if len(results[0].keypoints) > 0:
            keypoints = results[0].keypoints
            confs = keypoints.conf[0].tolist()  # 推論結果:1に近いほど信頼度が高い
            xys = keypoints.xy[0].tolist()  # 座標

            # 姿勢情報をCSVファイルに書き出す
            write_pose_to_csv(csv_path, frame_count, xys, confs)
            # キーポイントと骨格をフレームに描画する
            frame = draw_keypoints(frame, xys, confs)

        # フレームに骨格情報を描画したものを動画に書き出す
        video_writer.write(frame)
        frame_count += 1

    capture.release()
    video_writer.release()
    cv2.destroyAllWindows()

# Y05.7.3
# 関数として定義しておく

def get_pose(file_name:str, extension:str):
  ### Y05.1 今回はfpsを30で統一する
  # # ドラッグ&ドロップの場合
  # !ffmpeg -y -i {file_name}{extension} -vf "fps=30" {file_name}"_30fps.mp4"
  # # Google Driveの場合
  !ffmpeg -y -i "/content/drive/MyDrive/20240903_video/"{file_name}{extension} -vf "fps=30" {file_name}"_30fps.mp4"

  ### Y05.2 動画のwidth/height/fpsを取得
  import cv2

  video_path = file_name+extension
  cap = cv2.VideoCapture(video_path)
  width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fps = cap.get(cv2.CAP_PROP_FPS)

  print(file_name+extension)
  print("width:{}, height:{}, fps:{}".format(width,height,fps))

  video_path = file_name+"_30fps.mp4"
  cap = cv2.VideoCapture(video_path)
  width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fps = cap.get(cv2.CAP_PROP_FPS)

  print("")
  print(file_name+"_30fps.mp4")
  print("width:{}, height:{}, fps:{}".format(width,height,fps))

  ### Y05.3 入力動画のパスと出力ファイルのパスを指定
  input_video_path = "/content/"+file_name+"_30fps.mp4"
  output_video_path = file_name+"_30fps_with_pose.mp4"
  csv_path = file_name+"_30fps_pose_keypoints.csv"

  ### Y05.4 動画を処理し、姿勢情報を取得してCSVと動画に保存
  process_video(input_video_path, output_video_path, csv_path)

  ### Y05.5 結果確認用動画の作成
  !ffmpeg -y -i {output_video_path} -vf scale=600:-2 {file_name}_30fps_with_pose.mov

  ### Y05.6 動画の再生
  from IPython.display import HTML
  from base64 import b64encode

  mp4 = open( file_name+"_30fps_with_pose.mov", 'rb').read()
  data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
  HTML(f"""
  <video width="50%" height="50%" controls>
        <source src="{data_url}" type="video/mp4">
  </video>""")

### Y06.2 0フレーム目を可視化
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def visualize_zero_frame(file_path:str):
  data = pd.read_csv(file_path)

  # 0フレームのデータをフィルタリングする
  frame_0_data = data[data['frame'] == 0]

  # キーポイントの名前のリスト（列名からスコアを除いたもの）
  keypoint_names = [col.split('_')[0] for col in frame_0_data.columns[1::3]]

  # 0フレームのすべてのキーポイントのx座標とy座標を抽出する
  x_coords = frame_0_data.iloc[0, 1::3]
  y_coords = frame_0_data.iloc[0, 2::3]

  # キーポイントの接続を定義（CSVのラベルに基づく）
  keypoints = {
      'nose': 0, 'eye(L)': 1, 'eye(R)': 2, 'ear(L)': 3, 'ear(R)': 4,
      'shoulder(L)': 5, 'shoulder(R)': 6, 'elbow(L)': 7, 'elbow(R)': 8,
      'wrist(L)': 9, 'wrist(R)': 10, 'hip(L)': 11, 'hip(R)': 12,
      'knee(L)': 13, 'knee(R)': 14, 'ankle(L)': 15, 'ankle(R)': 16
  }

  connections = [
      ('nose', 'eye(L)'), ('nose', 'eye(R)'), ('eye(L)', 'ear(L)'), ('eye(R)', 'ear(R)'),
      ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
      ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
      ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
      ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
      ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
  ]

  # y軸を反転させた状態で0フレームの散布図をプロットする
  plt.figure(figsize=(10, 8))

  # キーポイントをプロット
  plt.scatter(x_coords, y_coords, c='blue')

  # 各キーポイントの横に名前をプロットする
  for i, name in enumerate(keypoint_names):
      plt.text(x_coords[i], y_coords[i], name, fontsize=9, ha='right')

  # キーポイントを矢印で接続
  for connection in connections:
      x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
      x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
      dx, dy = x2 - x1, y2 - y1
      plt.arrow(x1, y1, dx, dy, head_width=15, head_length=25, fc='gray', ec='gray', length_includes_head=True)

  plt.xlabel('X Coordinates')
  plt.ylabel('Y Coordinates')
  plt.title('Scatter Plot of Keypoints for Frame 0')
  plt.grid(True)
  plt.gca().invert_yaxis()  # 原点を左上にするためにy軸を反転する
  plt.axis('equal')  # 縦軸と横軸のスケールを同じにする
  plt.show()

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def visualize_zero_frame_norm(file_path:str):
  data = pd.read_csv(file_path)

  # 0フレームのデータをフィルタリングする
  frame_0_data = data[data['frame'] == 0]

  # キーポイントの名前のリスト（列名からスコアを除いたもの）
  keypoint_names = [col.split('_')[0] for col in frame_0_data.columns[1::3]]

  # 0フレームのすべてのキーポイントのx座標とy座標を抽出する
  x_coords = frame_0_data.iloc[0, 1::3]
  y_coords = frame_0_data.iloc[0, 2::3]

  # キーポイントの接続を定義（CSVのラベルに基づく）
  keypoints = {
      'nose': 0, 'eye(L)': 1, 'eye(R)': 2, 'ear(L)': 3, 'ear(R)': 4,
      'shoulder(L)': 5, 'shoulder(R)': 6, 'elbow(L)': 7, 'elbow(R)': 8,
      'wrist(L)': 9, 'wrist(R)': 10, 'hip(L)': 11, 'hip(R)': 12,
      'knee(L)': 13, 'knee(R)': 14, 'ankle(L)': 15, 'ankle(R)': 16
  }

  connections = [
      # ('nose', 'eye(L)'), ('nose', 'eye(R)'), ('eye(L)', 'ear(L)'), ('eye(R)', 'ear(R)'),
      ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
      ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
      ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
      ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
      ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
  ]

  # y軸を反転させた状態で0フレームの散布図をプロットする
  plt.figure(figsize=(10, 8))

  # キーポイントをプロット
  plt.scatter(x_coords, y_coords, c='blue')

  # 各キーポイントの横に名前をプロットする
  for i, name in enumerate(keypoint_names):
      plt.text(x_coords[i], y_coords[i], name, fontsize=9, ha='right')

  # キーポイントを矢印で接続
  for connection in connections:
      x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
      x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
      dx, dy = x2 - x1, y2 - y1
      plt.arrow(x1, y1, dx, dy, head_width=15, head_length=25, fc='gray', ec='gray', length_includes_head=True)

  # 単位ベクトルを計算する
  unit_vectors = []
  for connection in connections:
      x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
      x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
      dx, dy = x2 - x1, y2 - y1
      norm = np.sqrt(dx**2 + dy**2)
      if norm != 0:
          unit_vectors.append((dx / norm, dy / norm, x1, y1))

  # 単位ベクトルをプロットする
  for uv in unit_vectors:
      plt.arrow(uv[2], uv[3], uv[0] * 100, uv[1] * 100, head_width=5, head_length=10, fc='red', ec='red', length_includes_head=True)

  plt.xlabel('X Coordinates')
  plt.ylabel('Y Coordinates')
  plt.title('Visualization of Unit Vectors for Frame 0')
  plt.grid(True)
  plt.gca().invert_yaxis()  # 原点を左上にするためにy軸を反転する
  plt.axis('equal')  # 縦軸と横軸のスケールを同じにする
  plt.show()

### Y06.4 全てのフレームを単位ベクトルに変換してcsvファイルに格納

import pandas as pd
import numpy as np

# 単位ベクトルを計算し、結果を連結する関数を定義
def calc_and_concat_norms(cols, data):
    unit_vectors_list = []
    for col1, col2 in cols:
        # 各点のconfidenceを取得
        confidence_col1 = data[f'{col1}_score'].values
        confidence_col2 = data[f'{col2}_score'].values

        # ベクトルの差分を計算し、confidenceが0.1以下の場合は差分を0に設定
        vector_diff = np.where((confidence_col1 > 0.1)[:, None] & (confidence_col2 > 0.1)[:, None],
                               data[[f'{col2}_x', f'{col2}_y']].values - data[[f'{col1}_x', f'{col1}_y']].values,
                               np.array([0, 0]))

        # 単位ベクトルを計算
        norm_diff = np.linalg.norm(vector_diff, axis=1, keepdims=True)
        unit_vector_diff = np.where(norm_diff != 0, vector_diff / norm_diff, np.array([0, 0]))

        # 結果をデータフレームに変換
        unit_vectors_df = pd.DataFrame({
            f'unit_x_{col1}_{col2}': unit_vector_diff[:, 0],
            f'unit_y_{col1}_{col2}': unit_vector_diff[:, 1]
        })

        # 各ペアの結果をリストに追加
        unit_vectors_list.append(unit_vectors_df)

    # 全てのデータフレームを列方向に連結
    final_df = pd.concat(unit_vectors_list, axis=1)
    return final_df

# 指定されたディレクトリ内のファイルを処理する関数を定義
def transform_to_norm(filename: str):
    # ファイルパスを設定
    file_path = f"{filename}_30fps_pose_keypoints.csv"
    output_file_path = f"{filename}_30fps_pose_keypoints_norm.csv"

    # CSVファイルを読み込む
    data = pd.read_csv(file_path).fillna(0)

    # 対象の点のペアを定義（0フレーム目の可視化に基づく）
    points_pairs = [
        # ('nose', 'eye(L)'), ('nose', 'eye(R)'), ('eye(L)', 'ear(L)'), ('eye(R)', 'ear(R)'),
        ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
        ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
        ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
        ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
        ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
    ]

    # 単位ベクトルを計算し連結
    result_df = calc_and_concat_norms(points_pairs, data)

    # 結果をCSVファイルに保存
    result_df.to_csv(output_file_path, index=False)


### Y06.5 変換結果の可視化

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

def visualize_norms(file_name: str):
    # CSVファイルを読み込む
    file_path = file_name + '_30fps_pose_keypoints.csv'
    data = pd.read_csv(file_path)

    # 単位ベクトルが計算された結果のCSVファイルを読み込む
    norm_file_path = file_name + '_30fps_pose_keypoints_norm.csv'
    norm_data = pd.read_csv(norm_file_path)

    # フレーム数を取得
    num_frames = norm_data.shape[0]

    # カラーマップを設定（半透明にする）
    colors = cm.viridis(np.linspace(0, 1, num_frames))
    colors[:, 3] = 0.2  # アルファ値を設定（0.5にする）

    # 初期フレームのデータをフィルタリングする
    frame_0_data = data[data['frame'] == 0]

    # キーポイントの名前のリスト（列名からスコアを除いたもの）
    keypoint_names = [col.split('_')[0] for col in frame_0_data.columns[1::3]]

    # 初期フレームのすべてのキーポイントのx座標とy座標を抽出する
    x_coords = frame_0_data.iloc[0, 1::3]
    y_coords = frame_0_data.iloc[0, 2::3]

    # キーポイントの接続を定義（CSVのラベルに基づく）
    keypoints = {
        'nose': 0, 'eye(L)': 1, 'eye(R)': 2, 'ear(L)': 3, 'ear(R)': 4,
        'shoulder(L)': 5, 'shoulder(R)': 6, 'elbow(L)': 7, 'elbow(R)': 8,
        'wrist(L)': 9, 'wrist(R)': 10, 'hip(L)': 11, 'hip(R)': 12,
        'knee(L)': 13, 'knee(R)': 14, 'ankle(L)': 15, 'ankle(R)': 16
    }

    connections = [
        ('nose', 'shoulder(L)'), ('nose', 'shoulder(R)'), ('shoulder(L)', 'elbow(L)'),
        ('shoulder(R)', 'elbow(R)'), ('elbow(L)', 'wrist(L)'), ('elbow(R)', 'wrist(R)'),
        ('shoulder(L)', 'shoulder(R)'), ('shoulder(L)', 'hip(L)'), ('shoulder(R)', 'hip(R)'),
        ('hip(L)', 'hip(R)'), ('hip(L)', 'knee(L)'), ('hip(R)', 'knee(R)'),
        ('knee(L)', 'ankle(L)'), ('knee(R)', 'ankle(R)')
    ]

    # y軸を反転させた状態で初期フレームの散布図をプロットする
    plt.figure(figsize=(10, 8))

    # 初期フレームのキーポイントをプロット
    plt.scatter(x_coords, y_coords, c='blue')

    # 各キーポイントの横に名前をプロットする
    for i, name in enumerate(keypoint_names):
        plt.text(x_coords[i], y_coords[i], name, fontsize=9, ha='right')

    # 初期フレームのキーポイントを矢印で接続
    for connection in connections:
        x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
        x2, y2 = x_coords[keypoints[connection[1]]], y_coords[keypoints[connection[1]]]
        dx, dy = x2 - x1, y2 - y1
        plt.arrow(x1, y1, dx, dy, head_width=15, head_length=25, fc='gray', ec='gray', length_includes_head=True)

    # 単位ベクトルをプロットする
    for connection in connections:
        unit_x_col = f'unit_x_{connection[0]}_{connection[1]}'
        unit_y_col = f'unit_y_{connection[0]}_{connection[1]}'
        if unit_x_col in norm_data.columns and unit_y_col in norm_data.columns:
            for i in range(num_frames):
                x1, y1 = x_coords[keypoints[connection[0]]], y_coords[keypoints[connection[0]]]
                unit_x, unit_y = norm_data.iloc[i][unit_x_col], norm_data.iloc[i][unit_y_col]
                plt.arrow(x1, y1, unit_x * 100, unit_y * 100, head_width=5, head_length=10, fc=colors[i], ec=colors[i], length_includes_head=True)

    plt.xlabel('X Coordinates')
    plt.ylabel('Y Coordinates')
    plt.title('Visualization of Unit Vectors Over Time')
    plt.grid(True)
    plt.gca().invert_yaxis()  # 原点を左上にするためにy軸を反転する
    plt.axis('equal')  # 縦軸と横軸のスケールを同じにする
    plt.show()

### Y07.1 ライブラリをインストール
!pip install fastdtw
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw


def calc_dtw(a, b):
  x = np.array(pd.read_csv(a).fillna(0))
  y = np.array(pd.read_csv(b).fillna(0))
  distance, path = fastdtw(x, y, dist=euclidean)
  print(a, b, distance)

from sklearn.metrics import mean_squared_error

def calc_rmse(a, b):
  x = pd.read_csv(a).fillna(0)
  y = pd.read_csv(b).fillna(0)[:len(x)] #元動画の長さに合わせる
  rmse = np.sqrt(mean_squared_error(x, y))
  print(a, b,'{:.3f}'.format(rmse))